# 0. Install and Import Dependencies

In [16]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
import numpy as np

In [17]:
# Optional if you are using a GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# 1. Load Model

In [18]:
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

# 2. Make Detections

In [26]:
cap = cv2.VideoCapture('novak.mp4')
while cap.isOpened():
    ret, frame = cap.read()
    
    # Resize image
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384,640)
    input_img = tf.cast(img, dtype=tf.int32)
    
    # Detection section
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    
    # Render keypoints 
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.1)
    
    cv2.imshow('Movenet Multipose', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
keypoints_with_scores

array([[[7.1168280e-01, 5.8314019e-01, 3.1078106e-02],
        [7.1184385e-01, 5.8111113e-01, 3.4281317e-02],
        [7.1153420e-01, 6.0230774e-01, 7.9845833e-03],
        [7.3538387e-01, 5.8000135e-01, 5.6709841e-02],
        [7.5665921e-01, 6.2307823e-01, 3.1508242e-03],
        [7.5044984e-01, 5.3343135e-01, 2.1908937e-02],
        [7.5681061e-01, 6.4546603e-01, 1.6382450e-03],
        [7.5208133e-01, 4.8462430e-01, 1.7035991e-02],
        [7.5996935e-01, 6.4332259e-01, 7.0459116e-03],
        [7.2459477e-01, 4.8232728e-01, 1.0579231e-02],
        [7.1971333e-01, 6.2511826e-01, 1.6672703e-02],
        [7.7769703e-01, 5.2314067e-01, 1.0262468e-01],
        [7.7741235e-01, 5.8498234e-01, 2.0535205e-02],
        [7.8522575e-01, 4.7247329e-01, 1.1155416e-02],
        [7.8368688e-01, 5.5127513e-01, 6.0130600e-03],
        [7.9783291e-01, 4.7685885e-01, 6.6279843e-03],
        [7.8551501e-01, 5.3394407e-01, 2.0842087e-02]],

       [[6.7897123e-01, 1.6260698e-01, 1.6760480e-02],
        

In [ ]:
32*12

384

In [ ]:
# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

# 3. Draw Keypoints

In [ ]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)

# 4. Draw Edges

In [ ]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [ ]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)